**Exercício Prático: Avaliação de Classificadores KNN com K-Fold Cross-Validation - Dataset Vinhos (UCI)**

Leitura dos dados:

In [1]:
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

Pré-processamento dos Dados:

In [3]:
# Separar as variáveis
X = vinhos.drop('classe', axis=1)
y = vinhos['classe']

Configuração do Experimento:

In [4]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=None)

# Função experimentos
def executar_experimento(random_state, k_values):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state, stratify=y)

    resultados = []
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        acuracia = accuracy_score(y_test, y_pred)
        resultados.append({'k': k, 'acuracia': acuracia})

    return resultados


# Realizar os experimentos
random_states = [42, 17, 24]
k_values = [3, 5]

for random_state in random_states:
    resultados = executar_experimento(random_state, k_values)
    print(f"Resultados para random_state = {random_state}:")
    for resultado in resultados:
        print(f"  k = {resultado['k']}: Acurácia = {resultado['acuracia']}")
    print("-" * 20)

Resultados para random_state = 42:
  k = 3: Acurácia = 0.6851851851851852
  k = 5: Acurácia = 0.7222222222222222
--------------------
Resultados para random_state = 17:
  k = 3: Acurácia = 0.7592592592592593
  k = 5: Acurácia = 0.7592592592592593
--------------------
Resultados para random_state = 24:
  k = 3: Acurácia = 0.7037037037037037
  k = 5: Acurácia = 0.6851851851851852
--------------------


Estrutura de Cross-Validation com KFold:

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Estrutura de Cross-Validation com KFold:
def executar_experimento_kfold(random_state, k_values):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    resultados_kfold = []

    for k in k_values:
        acuracias = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            acuracia = accuracy_score(y_test, y_pred)
            acuracias.append(acuracia)

        media_acuracia = np.mean(acuracias)
        desvio_padrao_acuracia = np.std(acuracias)
        resultados_kfold.append({'k': k, 'media_acuracia': media_acuracia, 'desvio_padrao': desvio_padrao_acuracia})
    return resultados_kfold


# Realizar os experimentos com KFold
random_states = [42, 17, 24]
k_values = [3, 5]

for random_state in random_states:
    resultados_kfold = executar_experimento_kfold(random_state, k_values)
    print(f"Resultados KFold para random_state = {random_state}:")
    for resultado in resultados_kfold:
        print(f"  k = {resultado['k']}: Média da Acurácia = {resultado['media_acuracia']:.4f}, Desvio Padrão = {resultado['desvio_padrao']:.4f}")
    print("-" * 20)

Resultados KFold para random_state = 42:
  k = 3: Média da Acurácia = 0.7036, Desvio Padrão = 0.1392
  k = 5: Média da Acurácia = 0.6641, Desvio Padrão = 0.1001
--------------------
Resultados KFold para random_state = 17:
  k = 3: Média da Acurácia = 0.6987, Desvio Padrão = 0.1040
  k = 5: Média da Acurácia = 0.6876, Desvio Padrão = 0.1121
--------------------
Resultados KFold para random_state = 24:
  k = 3: Média da Acurácia = 0.7127, Desvio Padrão = 0.0968
  k = 5: Média da Acurácia = 0.7127, Desvio Padrão = 0.0940
--------------------


Análise dos Resultados:

In [8]:
melhores_resultados = {}
for random_state in random_states:
    resultados_kfold = executar_experimento_kfold(random_state, k_values)
    melhor_k = None
    melhor_acuracia = -1
    for resultado in resultados_kfold:
        if resultado['media_acuracia'] > melhor_acuracia:
            melhor_acuracia = resultado['media_acuracia']
            melhor_k = resultado['k']
        elif resultado['media_acuracia'] == melhor_acuracia:
            melhor_k = "Empate entre k=3 e k=5" # Registrando o empate
    melhores_resultados[random_state] = (melhor_k, melhor_acuracia)

for random_state, (melhor_k, melhor_acuracia) in melhores_resultados.items():
    print(f"Melhor k para random_state={random_state}: k={melhor_k}, Acurácia Média={melhor_acuracia:.4f}")

Melhor k para random_state=42: k=3, Acurácia Média=0.7036
Melhor k para random_state=17: k=3, Acurácia Média=0.6987
Melhor k para random_state=24: k=Empate entre k=3 e k=5, Acurácia Média=0.7127
